In [16]:
#this notebook takes the scraped wikipedia data (JSons) and
#transforms it into a dataframe for guessing

In [5]:
import json
import os
import numpy as np
from collections import Counter

binary_feature_counts = Counter()

unames = Counter()
jsns = filter(lambda s: s.startswith("grab_musicians"),os.listdir("musical_scraper"))
for fname in jsns:
    with open(os.path.join("musical_scraper",fname)) as fin:
        js = json.load(fin)
        for entity in js.values():
            unames[entity['_page_url']]+=1
            for key in entity.keys():
                binary_feature_counts[key]+=1


In [6]:
categorical_features = {
                        'Genres':[u'Genres',u'Genre(s)',u'Stylistic origins',u'Cultural origins'],
                        'Origin':[u'Origin',u'Country',],

                        'Occupation':[ u'Occupation(s)',u'Occupation',],
                        'Instruments':[u'Instruments',u'Typical instruments',]
    
                        
                       }

In [7]:
binary_feature_counts["_page_url"] = 0
binary_feature_counts["_list"] = 0
binary_feature_counts["Years active"] = 0

binary_feature_counts["Associated acts"] = 0

for name,f_list in categorical_features.items():
    for feature in f_list:
        binary_feature_counts[feature] = 0
        
binary_features = dict(binary_feature_counts.most_common(100)).keys()

In [8]:
from collections import defaultdict,Counter
feature_factor_freqs = defaultdict(int)

for fname in jsns:
    with open(os.path.join("musical_scraper",fname)) as fin:
        js = json.load(fin)
        
        
        for entity in js.values():
            attributes = set()
            
            for feature_name in binary_features:
                if feature_name in entity:
                    attributes.add(feature_name+':is_known')
            
            for feature_name,keys in categorical_features.items():
                for key in keys:
                    
                    vals = entity.get(key,'').lower().replace(',',' ')
                    vals = filter(len,vals.split())
                    
                    for val in vals:
                        
                        attributes.add(feature_name+":"+val)
            
            
            #_list
            list_url = entity.get("_list",'')
            category = list_url.split('/')[-1]
            
            
            attributes.add("category:"+category)
            
            #activity
            
            yrs_active = entity.get(u'Years active','').lower()
            yrs_active = yrs_active.replace(',',' ').replace(u'\u2013',' ').replace("-",' ').replace("present","2016")
            
            yrs_active = filter(len,yrs_active.split())
            yrs_active = filter(lambda word: word.isdigit() and len(word)==4
                                ,yrs_active)

            if len(yrs_active) >=2:
                start,end = int(yrs_active[0]), int(yrs_active[-1])
                
                for decade in np.arange(start//10,end//10+1)*10:
                    if decade < 1920:
                        decade = "before_1920"
                        
                    attributes.add("decades_active:"+str(decade))
                    
                attributes.add("first_activity:"+str(start//3*3))
                if end == 2016:
                    end = "still_active"
                else:
                    end = end//3*3
                attributes.add("last_activity:"+str(end))
            
            for attr in attributes:
                feature_factor_freqs[attr]+=1
            

In [9]:
feature_factor_freqs = Counter(
                            dict(
                                filter( lambda (key,val): feature_factor_freqs[key]>500, 
                                       feature_factor_freqs.items())
                                )
                             )


In [10]:
accepted_features = feature_factor_freqs.keys()
feature_to_id = {f:i for i,f in enumerate(accepted_features)}
len(accepted_features)

433

In [11]:
from collections import defaultdict,Counter
feature_factor_freqs = defaultdict(int)


rows = []

for fname in jsns:
    with open(os.path.join("musical_scraper",fname)) as fin:
        js = json.load(fin)
        
        
        for entity in js.values():
            attributes = set()
            
            for feature_name in binary_features:
                if feature_name in entity:
                    attributes.add(feature_name+':is_known')
            
            for feature_name,keys in categorical_features.items():
                for key in keys:
                    
                    vals = entity.get(key,'').lower().replace(',',' ')
                    vals = filter(len,vals.split())
                    
                    for val in vals:
                        
                        attributes.add(feature_name+":"+val)
            
            
            #_list
            list_url = entity.get("_list",'')
            category = list_url.split('/')[-1]
            
            
            attributes.add("category:"+category)
            
            #activity
            
            yrs_active = entity.get(u'Years active','').lower()
            yrs_active = yrs_active.replace(',',' ').replace(u'\u2013',' ').replace("-",' ').replace("present","2016")
            
            yrs_active = filter(len,yrs_active.split())
            yrs_active = filter(lambda word: word.isdigit() and len(word)==4
                                ,yrs_active)

            if len(yrs_active) >=2:
                start,end = int(yrs_active[0]), int(yrs_active[-1])
                
                for decade in np.arange(start//10,end//10+1)*10:
                    if decade < 1920:
                        decade = "before_1920"
                        
                    attributes.add("decades_active:"+str(decade))
                    
                attributes.add("first_activity:"+str(start//3*3))
                if end == 2016:
                    end = "still_active"
                else:
                    end = end//3*3
                attributes.add("last_activity:"+str(end))
            
            attribute_ids = filter(lambda x:x is not None, 
                                           map(feature_to_id.get, attributes)
                                   )
            row = np.zeros(len(accepted_features),dtype=np.bool)
            row[attribute_ids] = True
            rows.append(row)

In [12]:
accepted_features = map(lambda s: s.encode('utf8'),accepted_features)

In [13]:
import pandas as pd
df = pd.DataFrame(rows,columns = accepted_features)

In [14]:
df.to_csv("musicians_and_others.csv")

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline


plt.figure(figsize=[20,5])
_=plt.plot(*zip(*feature_factor_freqs["first_activity"].items()))
plt.xlim(1500,2100)
#feature_factor_freqs['Country']

AttributeError: 'int' object has no attribute 'items'